# Custom Training with TensorFlow in Sagemaker

# Download Data

In [1]:
%matplotlib inline

import os
import tarfile
import urllib
import shutil
import json
import random
import numpy as np
import tensorflow as tf
import sagemaker

from PIL import Image
from matplotlib import pyplot as plt

urls = ['http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz',
        'http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz']

print('Libraries imported')


Libraries imported


In [2]:
def download_and_extract(data_dir, download_dir):
    for url in urls:
        target_file = url.split('/')[-1]
        if target_file not in os.listdir(download_dir):
            print('Downloading', url)
            urllib.request.urlretrieve(url, os.path.join(download_dir, target_file))
            tf = tarfile.open(url.split('/')[-1])
            tf.extractall(data_dir)
        else:
            print('Already downloaded', url)

def get_annotations(file_path, annotations={}):
    
    with open(file_path, 'r') as f:
        rows = f.read().splitlines()

    for i, row in enumerate(rows):
        image_name, _, _, _ = row.split(' ')
        class_name = image_name.split('_')[:-1]
        class_name = '_'.join(class_name)
        image_name = image_name + '.jpg'
        
        annotations[image_name] = 'cat' if class_name[0] != class_name[0].lower() else 'dog'
    
    return annotations

In [3]:
if not os.path.isdir('data'):
    os.mkdir('data')

download_and_extract('data', '.')

# Dataset for Training

In [4]:
annotations = get_annotations('data/annotations/trainval.txt')
annotations = get_annotations('data/annotations/test.txt', annotations)

total_count = len(annotations.keys())
print('Total examples', total_count)

Total examples 7349


In [8]:
next(iter(annotations.items()))

('Abyssinian_100.jpg', 'cat')

In [5]:
classes = ['cat', 'dog']
sets = ['train', 'validation']
root_dir = 'custom_data'

if not os.path.isdir(root_dir):
    os.mkdir(root_dir)
    
for set_name in sets:
    if not os.path.isdir(os.path.join(root_dir, set_name)):
        os.mkdir(os.path.join(root_dir, set_name))
    for class_name in classes:
        folder = os.path.join(root_dir, set_name, class_name)
        if not os.path.isdir(folder):
            os.mkdir(folder)

Copy the files to correct set/ class folders

In [9]:
for image, class_name in annotations.items():
    target_set = 'validation' if random.randint(0, 99) < 20 else 'train'
    target_path = os.path.join(root_dir, target_set, class_name, image)
    shutil.copy(os.path.join('data/images/', image), target_path)

In [10]:
sets_counts = {
    'train': 0,
    'validation': 0
}

for set_name in sets:
    for class_name in classes:
        path = os.path.join(root_dir, set_name, class_name)
        count = len(os.listdir(path))
        print(path, 'has', count, 'images')
        sets_counts[set_name] += count

print(sets_counts)

custom_data/train/cat has 1891 images
custom_data/train/dog has 3978 images
custom_data/validation/cat has 480 images
custom_data/validation/dog has 1000 images
{'train': 5869, 'validation': 1480}


# Training Script - Create Model

In [32]:
%%writefile train.py
import tensorflow as tf
import argparse
import os
import json

def create_model():
    model = tf.keras.model.Sequential([
        tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                                          pooling='avg', weight='imagenet',
                                          input_shape=(128,128,3)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1,activation='sigmoid')
    ])
    model.layers[0].trainable = False
    model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
    return model

Overwriting train.py


# Training Script - Data Generators

In [33]:
%%writefile -a train.py

def create_data_generators(root_dir, batch_size):
    train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
        horizontal_flip = True,
        zoom_range=[0.8,1.2],
        rotation_range=20
    ).flow_from_directory(
    os.path.join(root_dir, 'train'),
    target_size = (128,128),
    batch_size=batch_size,
    class_mode='binary'
    )
    
    val_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
    ).flow_from_directory(
    os.path.join(root_dir, 'validation'),
    target_size = (128,128),
    batch_size=batch_size,
    class_mode='binary'
    )
    return train_data_generator, val_data_generator
    

Appending to train.py


# Training Script - Putting it Together

In [34]:
%%writefile -a train.py

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parse.add_argument('--epochs', type=int, default=3)
    parse.add_argument('--batch_size', type=int, default=16)
    parse.add_argument('--steps', type=int, default=int(5869/16))
    parse.add_argument('--val_steps', type=int, default=int(1480/16))
    
    parser.add_argument('--model_dir',type = str)
    parser.add_argument('--sm-modle-ir',type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train',type=str, default=os.environ.get('SM_CHANNEL_TRAINING'))  
    
    args, _ = parser.parse_known_args()
    
    local_output_dir = args.sm_model_dir
    local_root_dir = args.train
    batch_size = args.batch_size
    
    model = create_model()
    train_gen, val_gen = create_data_generators(local_root_dir, batch_size)
    
    _ = model.fit(
        train_gen,
        epochs=args.epochs,
        steps_per_epoch=args.steps,
        validation_data=val_gen,
        validation_steps=args.val_steps
    )
    
    model.save(os.path.join(local_output_dir, 'model', '1'))
        

Appending to train.py


# Upload Dataset to S3

In [17]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name = 'petcustomck'

print('Uploading Data in S3...')
s3_data_path = sess.upload_data(path=root_dir, bucket=bucket_name, key_prefix='data')
print('Uploaded to', s3_data_path)

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


Uploading Data in S3...
Uploaded to s3://petcustomck/data


# Train with TensorFlow Estimator

In [45]:
from sagemaker.tensorflow import TensorFlow

pets_estimator = TensorFlow(
        entry_point='train.py',
        role=role,
        train_instance_type='ml.m5.large',
        train_instance_count=1,
        framework_version='2.1.0',
        py_version='py3',
        output_path='s3://petcustomck/'
)

In [46]:
pets_estimator.fit(s3_data_path)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-06-09 21:02:44 Starting - Starting the training job...
2020-06-09 21:02:47 Starting - Launching requested ML instances.........
2020-06-09 21:04:17 Starting - Preparing the instances for training...
2020-06-09 21:04:57 Downloading - Downloading input data......
2020-06-09 21:06:11 Training - Downloading the training image...
2020-06-09 21:06:23 Training - Training image download completed. Training in progress.2020-06-09 21:06:28,845 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-09 21:06:28,851 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-09 21:06:29,249 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-09 21:06:29,266 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-09 21:06:29,281 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-09 21:06:29,291 sagemaker-containers INFO     Invoking u

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 96/366 [======>.......................] - ETA: 1:20 - loss: 0.5678 - accuracy: 0.7462#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/366 [======>.......................] - ETA: 1:20 - loss: 0.5704 - accuracy: 0.7463#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015191/366 [==============>...............] - ETA: 49s - loss: 0.4824 - accuracy: 0.7838#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015192/366 [==============>...............] - ETA: 48s - loss: 0.4813 - accuracy: 0.7843#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015287/366 [======================>.......] - ETA: 21s - loss: 0.4277 - accuracy: 0.8082#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015288/366 [======================>.......] - ETA: 21s - loss: 0.4281 - accuracy: 0.8080#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

...

#015  1/366 [..............................] - ETA: 2:16 - loss: 0.5186 - accuracy: 0.8125#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  2/366 [..............................] - ETA: 1:56 - loss: 0.3616 - accuracy: 0.8750#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  3/366 [..............................] - ETA: 1:46 - loss: 0.3318 - accuracy: 0.8958#010#010#010#010#010#010#010#010#010#010#0

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 96/366 [======>.......................] - ETA: 1:06 - loss: 0.2372 - accuracy: 0.8939#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 97/366 [======>.......................] - ETA: 1:06 - loss: 0.2356 - accuracy: 0.8943#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

.....] - ETA: 43s - loss: 0.2371 - accuracy: 0.8962#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015192/366 [==============>...............] - ETA: 43s - loss: 0.2364 - accuracy: 0.8967#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015193/366 [==============>...............] - ETA: 42s - loss: 0.2369 - accuracy: 0.8959#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01

7/366 [======================>.......] - ETA: 19s - loss: 0.2322 - accuracy: 0.8993#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015288/366 [======================>.......] - ETA: 19s - loss: 0.2318 - accuracy: 0.8995#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015289/366 [======================>.......] - ETA: 18s - loss: 0.2315 - accuracy: 0.8996#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01

2020-06-09 21:12:19.705960: W tensorflow/python/util/util.cc:319] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /opt/ml/model/model/1/assets
INFO:tensorflow:Assets written to: /opt/ml/model/model/1/assets
[2020-06-09 21:12:27.365 ip-10-0-144-126.us-east-2.compute.internal:22 INFO utils.py:25] The end of training job file will not be written for jobs running under SageMaker.
2020-06-09 21:12:27,895 sagemaker-containers INFO     Reporting training SUCCESS

2020-06-09 21:12:39 Uploading - Uploading generated training model
2020-06-09 21:12:39 Completed - Training job completed
Training seconds: 462
Billable seconds: 462


# Deploy TensorFlow Model

In [48]:
pets_predictor = pets_estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large')
print('\nModel is Deployed')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
Using already existing model: tensorflow-training-2020-06-09-21-02-42-623


-------------!
Model is Deployed


# Final Predictions

In [49]:
cat_dir = 'custom_data/validation/cat/'
cat_images = [os.path.join(cat_dir, x) for x in os.listdir(cat_dir)]
print(cat_images[0])

dog_dir = 'custom_data/validation/dog/'
dog_images = [os.path.join(dog_dir, x) for x in os.listdir(dog_dir)]
print(dog_images[0])

custom_data/validation/cat/Sphynx_162.jpg
custom_data/validation/dog/basset_hound_58.jpg


In [50]:
def get_pred(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    img = np.expand_dims(img, axis=0)

    results = pets_predictor.predict(img)
    return results

In [ ]:
image_path = cat_images[0]
results = get_pred(image_path)

print(results)

In [ ]:
class_id = int(np.squeeze(results['predictions']) > 0.5)
print('Predicted class_id:', class_id, 'with class_name:', classes[class_id])

# Delete Model Endpoint

In [ ]:
sagemaker_session.delete_endpoint(pets_predictor.endpoint)